In [1]:
%matplotlib ipympl
# %matplotlib qt

import sys
import time
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import NMF, PCA
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
from ardal import Ardal
import _ardal


def makePCA(matrix, categories, D3=True, Dplus=False):
    """
    Performs PCA on a matrix and visualizes the results with a legend.

    Args:
        matrix (pd.DataFrame): The input matrix (rows are samples, columns are features).
        categories (dict): A dictionary mapping matrix indices (sample IDs) to categories.
        D3 (bool): Whether to create a 3D plot (default: True).
        Dplus (bool): Whether to use a size map (default: False).

    Raises:
        ValueError: If the matrix is not a Pandas DataFrame or if categories is not a dictionary.
    """

    # Input validation
    if not isinstance(matrix, pd.DataFrame):
        raise ValueError("matrix must be a Pandas DataFrame.")
    if not isinstance(categories, dict):
        raise ValueError("categories must be a dictionary.")

    # Color setup
    unique_categories = sorted(set(categories.values()))
    cmap = sns.color_palette("Spectral", as_cmap=True)
    colours = [cmap(i) for i in np.linspace(0, 1, len(unique_categories))]
    cdict = dict(zip(unique_categories, colours))

    matrix_indices = matrix.index
    matrix_standardized = StandardScaler().fit_transform(matrix)

    n = min(len(matrix_indices), 20)
    pca = PCA(n_components=n)
    pca_result = pca.fit_transform(matrix_standardized)
    pca_df = pd.DataFrame(data=pca_result[:, :4], columns=['PC1', 'PC2', 'PC3', 'PC4'], index=matrix_indices)

    # Plotting
    if D3:
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')

        if Dplus:
            min_val = pca_df["PC4"].min()
            max_val = pca_df["PC4"].max()
            sizemap = 100 + ((pca_df['PC4'] - min_val) * 100) / (max_val - min_val)
            scatter = ax.scatter(pca_df['PC1'], pca_df['PC2'], pca_df['PC3'], s=sizemap, alpha=0.8)
        else:
            for category in unique_categories:
                indices = [idx for idx, cat in categories.items() if cat == category]
                subset = pca_df.loc[indices]
                scatter = ax.scatter(subset['PC1'], subset['PC2'], subset['PC3'], alpha=0.8, c=[cdict[category]] * len(subset), label=category)

            ax.legend()

        ax.set_zlabel('PC3')

    else:
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111)

        if Dplus:
            min_val = pca_df['PC3'].min()
            max_val = pca_df['PC3'].max()
            sizemap = 200 + ((pca_df['PC3'] - min_val) * 50) / (max_val - min_val)
            scatter = ax.scatter(pca_df['PC1'], pca_df['PC2'], s=sizemap, alpha=0.7)
        else:
            for category in unique_categories:
                indices = [idx for idx, cat in categories.items() if cat == category]
                subset = pca_df.loc[indices]
                scatter = ax.scatter(subset['PC1'], subset['PC2'], alpha=0.8, c=[cdict[category]] * len(subset), label=category)

            ax.legend()

    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.grid(True)

    plt.show()



def cluster(dist_matrix, nclusters):
    from sklearn.cluster import KMeans

    dist_array = dist_matrix.values

    # Perform k-means clustering
    kmeans = KMeans(n_clusters=nclusters, random_state=0, n_init='auto')  # Set random_state for reproducibility
    cluster_labels = kmeans.fit_predict(dist_array)

    # Create the cluster membership dictionary
    cluster_membership = {}
    for i, guid in enumerate(dist_matrix.index):
        cluster_membership[guid] = cluster_labels[i]

    return cluster_membership
    


# data = ["./data/Cparv_matrix.npy", "./data/Cparv_headers.json"]
# ard = Ardal(data)
# ard.stats()

In [2]:
import numpy as np
import pandas as pd
import json

def create_test_dataset():
    """
    Creates a simulated test dataset for testing the Ardal.unique functionality.

    Returns:
        tuple: A tuple containing the matrix (NumPy array) and headers (dictionary).

    "SNP1" is unique to "GUID1" and "GUID2".
    "SNP2" is unique to "GUID3", "GUID4", and "GUID5".
    "SNP3" is unique to "GUID6" and "GUID7".
    "SNP4" is unique to "GUID8", "GUID9", and "GUID10".
    "SNP5" is present in all GUIDs.
    "SNP6" is present in GUID1, GUID2, GUID3, GUID4, GUID5, GUID6, GUID7.
    "SNP7" is present in GUID1, GUID2, GUID3, GUID4, GUID5, GUID6, GUID7, GUID8, GUID9, GUID10.
    "SNP8" is present in GUID1, GUID2, GUID3, GUID4, GUID5.
    "SNP9" is present in GUID6, GUID7, GUID8, GUID9, GUID10.
    "SNP10" is present in GUID1, GUID2, GUID3, GUID4, GUID5, GUID6, GUID7, GUID8, GUID9.
    """

    # Define GUIDs and SNPs
    guids = [f"GUID{i}" for i in range(1, 11)]  # GUID1 to GUID10
    alleles = [f"SNP{i}" for i in range(1, 11)]  # SNP1 to SNP10

    # Create an empty matrix (all zeros)
    matrix = np.zeros((len(guids), len(alleles)), dtype=np.uint8)

    # Set unique SNPs
    matrix[0, 0] = 1  # SNP1 in GUID1
    matrix[1, 0] = 1  # SNP1 in GUID2
    matrix[2, 1] = 1  # SNP2 in GUID3
    matrix[3, 1] = 1  # SNP2 in GUID4
    matrix[4, 1] = 1  # SNP2 in GUID5
    matrix[5, 2] = 1  # SNP3 in GUID6
    matrix[6, 2] = 1  # SNP3 in GUID7
    matrix[7, 3] = 1  # SNP4 in GUID8
    matrix[8, 3] = 1  # SNP4 in GUID9
    matrix[9, 3] = 1  # SNP4 in GUID10

    # Set SNP5 to be present in all GUIDs
    matrix[:, 4] = 1  # SNP5 in all GUIDs

    # Set SNP6 to be present in GUID1, GUID2, GUID3, GUID4, GUID5, GUID6, GUID7
    matrix[0:7, 5] = 1

    # Set SNP7 to be present in all GUIDs
    matrix[:, 6] = 1

    # Set SNP8 to be present in GUID1, GUID2, GUID3, GUID4, GUID5
    matrix[0:5, 7] = 1

    # Set SNP9 to be present in GUID6, GUID7, GUID8, GUID9, GUID10
    matrix[5:10, 8] = 1

    # Set SNP10 to be present in GUID1, GUID2, GUID3, GUID4, GUID5, GUID6, GUID7, GUID8, GUID9
    matrix[0:9, 9] = 1

    # Create headers
    headers = {
        "guids": guids,
        "alleles": alleles
    }

    return matrix, headers

In [ ]:
ard.neighbourhood("SRR6147472_UKP3", 100)

In [ ]:
s = time.time()
b20_n = ard.neighbourhood("UKP215", 200)
e = time.time()

h_D = ard.pairwise(metric="hamming")

print(e-s)

In [ ]:
# dict(zip(ard._headers["guids"], cluster_assignments))
makePCA(h_D, cluster_assignments)

## CPP package

In [4]:
import _ardal
import json
import numpy as np

# d = 100000
# g = "SPT12274"

# matrix = np.ascontiguousarray(np.load("/home/amorris/BioInf/SSD/BioInf/Plasmodium_vcfs/Pf_SPT_matrix.npy"))
# with open("/home/amorris/BioInf/SSD/BioInf/Plasmodium_vcfs/Pf_SPT_headers.json", "r") as f:
#     headers = json.load(f)

# cpp_ard = _ardal.AlleleMatrix(matrix)

def encodeGuid( guid : str ):
    return headers["guids"].index(guid)

def decodeGuid( row_coord : int ):
    return headers["guids"][row_coord]

def encodeAllele( allele : str ):
    return headers["alleles"].index(allele)

def decodeAllele( col_coord : int ):
    return headers["alleles"][col_coord]

In [ ]:
guid_i = encodeGuid(g)

cpp_ard.neighbourhoodSIMD(guid_i, d)

## Python library

In [ ]:
from ardal import Ardal
import pandas as pd

# data = ["/home/amorris/BioInf/SSD/BioInf/Plasmodium_vcfs/Pf_matrix.npy", "/home/amorris/BioInf/SSD/BioInf/Plasmodium_vcfs/Pf_headers.json"]
data = ["/home/amorris/BioInf/burkholderia_WD/data/allele_matrices/BG_core_matrix.npy", "/home/amorris/BioInf/burkholderia_WD/data/allele_matrices/BG_core_headers.json"]
ard = Ardal(data)
ard.stats()
# df = pd.read_csv("/home/amorris/BioInf/ProtoDB/WD/Pf_SPT_hamming.csv",  index_col=0)
meta = pd.read_csv("/home/amorris/BioInf/burkholderia_WD/data/Mullins_Onion_Jones.csv")

In [6]:
matrix, headers = create_test_dataset()
ard = Ardal([matrix, headers])
cpp_ard = _ardal.AlleleMatrix(matrix)

alleles = ["SNP5", "SNP10"]
n = len(alleles)
allele_coords = [encodeAllele(allele) for allele in alleles]
input_coords = np.array([[encodeGuid(guid), allele] for guid in ard.getHeaders()["guids"] for allele in allele_coords])
access_result = zip(input_coords, cpp_ard.access(input_coords))
# print(list(result))
decoded_results = [decodeGuid(guid_c) for (guid_c, allele_c), r in access_result if r==1]

results = [guid for guid in set(decoded_results) if decoded_results.count(guid) == n]

print(results)

['GUID6', 'GUID9', 'GUID2', 'GUID8', 'GUID1', 'GUID3', 'GUID5', 'GUID7', 'GUID4']


In [ ]:
import time
d = 90000
g = "ERR1100643"
# ard.flushCache()
s = time.time()
ard_neigh = ard.neighbourhood(g, d, simd=True)
e = time.time()

ard_neigh
e-s

In [ ]:
cat = dict(zip(list(meta["SRA"]), list(meta["Clade"])))
clus = cluster(ardD_hamming, 10)
makePCA(ardD_hamming, clus, D3=True)

In [ ]:
guid_ids = [guid for guid, c in clus.items() if c in [5]]
print(guid_ids)
d1 = ard.unique(guid_ids)

d = {'GCA_902830895.1_ASM90283089v1_genomic.4394500.T',
 'GCA_902830905.1_ASM90283090v1_genomic.1733623.C',
 'GCA_902830905.1_ASM90283090v1_genomic.1759564.C',
 'GCA_902830905.1_ASM90283090v1_genomic.3025340.A',
 'GCA_902830905.1_ASM90283090v1_genomic.4319502.C',
 'GCA_902831285.1_ASM90283128v1_genomic.4462753.T',
 'GCA_902831285.1_ASM90283128v1_genomic.446892.C'}

d1

In [ ]:
d = 2000000
g = "SPT12274"
neigh = df.loc[g][df.loc[g] < d].to_dict()
ard_neigh = ard.neighbourhood(g, d, simd=True)

results_box = {"ardal" : ard_neigh, "hamming" : neigh}

results_df = pd.DataFrame(results_box).sort_values(by="ardal", ascending=True)
results_df = results_df.drop(g)

matching_guids = []
mismatching_guids = []

for guid in results_df.index:
    if results_df.loc[guid, "ardal"] == results_df.loc[guid, "hamming"]:
        matching_guids.append(guid)
    else:
        mismatching_guids.append(guid)

# print(bools)
print(f"MATCH : {len(matching_guids)}")
print(f"MISMATCH : {len(mismatching_guids)} {mismatching_guids}")

In [ ]:
results_df.loc["SPT43391"]
len(headers["alleles"])%32

In [ ]:

# num_samples = 100
# num_alleles = 50
# allele_matrix = np.random.randint(0, 2, (num_samples, num_alleles))

K = 4

X=ard.getMatrix()
nmf = NMF(n_components=K, init='random', random_state=12)
W = nmf.fit_transform(X)
H = nmf.components_

W = W/np.sum(W, axis=1)[:, np.newaxis]

cluster_assignments = np.argmax(H, axis=0)

fig, ax = plt.subplots(figsize=(10, 6))

bottom = np.zeros(ard.stats()["n_guids"])
for k in range(K):
    ax.bar(range(ard.stats()["n_guids"]), W[:, k], bottom=bottom, label=f'Ancestry {k+1}')
    bottom += W[:, k]

ax.set_xlabel('Sample')
ax.set_ylabel('Admixture Proportion')
ax.set_title(f'Admixture Proportions (K={K})')
ax.legend()

guids = ard.getHeaders()["guids"]
plt.xticks(range(len(guids)), guids, rotation=90)
plt.tight_layout()
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import NMF, PCA

# 1. Dimensionality Reduction (if necessary)
if W.shape[1] > 3:
    pca = PCA(n_components=3)  # Reduce to 3D for plotting
    W_reduced = pca.fit_transform(W)
elif W.shape[1] == 3:  # Already 3D
    W_reduced = W
else: # If W has fewer than 3 components, we'll pad it with zeros.
    W_reduced = np.pad(W, ((0,0), (0, 3 - W.shape[1])), mode='constant')



# 2. 3D Scatter Plot
# col_cat = np.argmax(W, axis=1)  # Get dominant ancestry for coloring
col_cat = np.max(W, axis=1)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d') # Create a 3D axes


scatter = ax.scatter(W_reduced[:, 0], W_reduced[:, 1], W_reduced[:, 2], c=col_cat, cmap='viridis')


ax.set_title("NMF Admixture 3D Scatter Plot")


# Set labels based on dimensionality reduction or original W matrix:
if W.shape[1] > 3:  # If PCA was used
    ax.set_xlabel("Principal Component 1")
    ax.set_ylabel("Principal Component 2")
    ax.set_zlabel("Principal Component 3")
elif W.shape[1] == 3:
    ax.set_xlabel("Ancestry Component 1")  # Adjust if you have specific names
    ax.set_ylabel("Ancestry Component 2")
    ax.set_zlabel("Ancestry Component 3")
else:
    ax.set_xlabel("Ancestry Component 1")
    ax.set_ylabel("Ancestry Component 2")
    ax.set_zlabel("Padding") # Indicate that the third dimension is padding


fig.colorbar(scatter, label="Dominant Ancestry")
plt.show()




In [ ]:
colors = ['red', 'green', 'blue', 'purple']

# Number of samples and ancestries
n_samples, n_ancestries = W.shape

# Create the admixture bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust figure size if needed

bottom = np.zeros(n_samples)  # Starting point for each bar segment

for k in range(n_ancestries):
    ax.bar(range(n_samples), W[:, k], bottom=bottom, color=colors[k % len(colors)], label=f"Ancestry {k+1}")
    bottom += W[:, k]  # Update the bottom for the next ancestry component


# Customize the plot
ax.set_xticks(range(n_samples))
ax.set_xticklabels(range(1, n_samples + 1))  # Or use sample names if available
ax.set_xlabel("Sample")
ax.set_ylabel("Admixture Proportion")
ax.set_title("Admixture Bar Plot")

ax.legend(title="Ancestries")
plt.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.show()


# Number of samples and ancestries
n_samples, n_ancestries = W.shape

# Create the admixture bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust figure size if needed

bottom = np.zeros(n_samples)  # Starting point for each bar segment

for k in range(n_ancestries):
    ax.bar(range(n_samples), W[:, k], bottom=bottom, color=colors[k % len(colors)], label=f"Ancestry {k+1}")
    bottom += W[:, k]  # Update the bottom for the next ancestry component


# Customize the plot
ax.set_xticks(range(n_samples))
ax.set_xticklabels(range(1, n_samples + 1))  # Or use sample names if available
ax.set_xlabel("Sample")
ax.set_ylabel("Admixture Proportion")
ax.set_title("Admixture Bar Plot")

ax.legend(title="Ancestries")
plt.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.show()



In [ ]:
print(np.max(W, axis=1))
W

In [ ]:
guids = list(b20_n.keys())
print(guids)
s = time.time()
core = ard.core(guids, 0.9)
e = time.time()

accessory = ard.accessory(guids, 0.9)

print(e-s)
print(len(core), len(accessory), len(core)+len(accessory))

d = defaultdict(list)
for c in core:
    d[c.split(".")[0]].append(c)